In [1]:
import numpy as np
import os
import PIL
import glob
from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
import sys
import PIL
import os
import tensorflow.keras.backend as K
import keras
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Dropout
from keras.layers import BatchNormalization, Flatten
from keras.layers import Activation
from keras.layers import Reshape # new! 
from keras.layers import Conv2DTranspose, UpSampling2D # new! 
from keras.optimizers import RMSprop # new!
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import time
import cv2

In [2]:
files = glob.glob('../data/dogs_cats/train/*.jpg')
cat_files = [x for x in files if 'cat' in x.split('/')[-1]]

In [3]:
src = Image.open(cat_files[0])

In [4]:
def get_discriminator():
    input_layer = Input((112, 112, 3))
    re_layer = keras.applications.ResNet50(include_top=False,
                                           weights = 'imagenet',
                                           input_shape = (112, 112, 3),
                                           pooling = 'avg')(input_layer)
    final_layer = Dense(1, activation='sigmoid')(re_layer)
    model = Model(inputs=input_layer, outputs = final_layer)
    return model

In [5]:
discriminator = get_discriminator()

2023-01-07 18:16:33.113454: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 18:16:33.158553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 18:16:33.158959: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 18:16:33.160127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
discriminator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 112, 112, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,589,761
Trainable params: 23,536,641
Non-trainable params: 53,120
_________________________________________________________________


In [7]:
discriminator.compile(
    loss = keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.SGD(),
    metrics = ['accuracy']
)

In [8]:
z_dimensions = 32

In [9]:
def build_generator(latent_dim=z_dimensions, 
                    depth=64, p=0.4):
    
    # Define inputs
    noise = Input((latent_dim,))
    
    # First dense layer
    dense1 = Dense(7*7*depth)(noise)
    dense1 = BatchNormalization(momentum=0.9)(dense1) # default momentum for moving average is 0.99
    dense1 = Activation(activation='relu')(dense1)
    dense1 = Reshape((7,7,depth))(dense1)
    dense1 = Dropout(p)(dense1)
    
    # De-Convolutional layers
    conv1 = UpSampling2D()(dense1)
    conv1 = Conv2DTranspose(int(depth/2), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv1)
    conv1 = BatchNormalization(momentum=0.9)(conv1)
    conv1 = Activation(activation='relu')(conv1)
    
    conv2 = UpSampling2D()(conv1)
    conv2 = Conv2DTranspose(int(depth/4), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv2)
    conv2 = BatchNormalization(momentum=0.9)(conv2)
    conv2 = Activation(activation='relu')(conv2)

    conv3 = UpSampling2D((4, 4))(conv2)
    conv3 = Conv2DTranspose(int(depth/4), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv3)
    conv3 = BatchNormalization(momentum=0.9)(conv3)
    conv3 = Activation(activation='relu')(conv3)
    
    conv4 = Conv2DTranspose(int(depth/8), 
                            kernel_size=5, padding='same', 
                            activation=None,)(conv3)
    conv4 = BatchNormalization(momentum=0.9)(conv4)
    conv4 = Activation(activation='relu')(conv4)

    # Output layer
    image = Conv2D(3, kernel_size=1, padding='valid', 
                   activation='sigmoid')(conv4)

    # Model definition    
    model = Model(inputs=noise, outputs=image)
    
    return model

In [10]:
generator = build_generator()

In [11]:
generator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_1 (Dense)             (None, 3136)              103488    
                                                                 
 batch_normalization (BatchN  (None, 3136)             12544     
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 3136)              0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 dropout (Dropout)           (None, 7, 7, 64)          0         
                                                           

In [12]:
z = Input(shape=(z_dimensions,))
img = generator(z)

In [13]:
discriminator.trainable = False

In [14]:
pred = discriminator(img)

In [15]:
adversarial_model = Model(z, pred)

In [16]:
adversarial_model.compile(loss='binary_crossentropy', 
                          optimizer=keras.optimizers.SGD(), 
                          metrics=['accuracy'])

In [17]:
pred

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'model')>

In [18]:
adversarial_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32)]              0         
                                                                 
 model_1 (Functional)        (None, 112, 112, 3)       190019    
                                                                 
 model (Functional)          (None, 1)                 23589761  
                                                                 
Total params: 23,779,780
Trainable params: 183,603
Non-trainable params: 23,596,177
_________________________________________________________________


In [19]:
def train(epochs = 200, batch = 32, z_dim = z_dimensions):
    d_metrics, a_metrics = [], []
    
    running_d_loss = 0
    running_d_acc = 0
    running_a_loss = 0
    running_a_acc = 0
    current_batch_start = 0
    count = 0
    for i in range(epochs):
        real_imgs = np.zeros(shape=(batch, 112, 112, 3))
        real_img_start = 0
        for x_ in range(current_batch_start, current_batch_start + batch):
            img = cv2.imread(cat_files[x_])
            img = cv2.resize(img, (112, 112))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255
            real_imgs[real_img_start] = img
            real_img_start += 1
        # break

        # Generate random images
        random_latent = np.random.uniform(
                0, 1.0, size=(batch, z_dimensions)
            )
        fake_imgs = generator.predict(
            random_latent, verbose = 1
        )
        x = np.concatenate((real_imgs, fake_imgs))
        # return real_imgs

        y = np.ones([2 * batch, 1])
        y[batch:, :] 0

        d_metrics.append(
            discriminator.train_on_batch(x, y)
        )

        running_d_loss += d_metrics[-1][0]
        running_d_acc += d_metrics[-1][1]

        noise = np.random.uniform(
            
        )
        break
    return x


In [20]:
data = train()

2023-01-07 18:20:17.127333: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


1/1 [==============================] - 2s 2s/step


2023-01-07 18:20:17.999625: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


In [21]:
data.shape

(64, 112, 112, 3)

In [25]:
x = np.ones([32 * 2, 1])

In [29]:
x[32:, :] = 0

In [30]:
x

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])